In [ ]:
# Ignorar os avisos
import warnings
warnings.filterwarnings('always')  # Configurar para mostrar todos os avisos
warnings.filterwarnings('ignore')  # Ignorar os avisos

# Visualização e manipulação de dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import missingno as msno
from scipy.stats import ttest_ind
from sklearn.datasets import fetch_openml  # Adicionada
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
# Configuração
style.use('fivethirtyeight')  # Estilo de plotagem
sns.set(style='whitegrid', color_codes=True)  # Configurações de estilo para Seaborn

# Importar algoritmos de modelagem necessários.

# Classificação.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Regressão
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# Seleção de modelo
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Pré-processamento
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer  # Importar o SimpleImputer do módulo sklearn.impute

# Métricas de avaliação
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error  # para regressão
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # para classificação



In [ ]:
df=pd.read_csv('analise_vinho_tinto.csv')

In [ ]:
df = df.iloc[:, 1:]

In [ ]:
df.shape

In [ ]:
df.columns 

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## Análise de Correlação das Características da Qualidade do Vinho

Através da aplicação das técnicas de correlação de Pearson e Spearman, podemos discernir as características que mais influenciam a qualidade do vinho. Os resultados indicam que as variáveis de álcool e densidade apresentam as maiores correlações. A análise de Spearman ainda revela o cloreto como uma variável com significativa correlação, enquanto a análise de Pearson destaca a acidez volátil. É fundamental notar, no entanto, que nenhuma dessas correlações ultrapassa o limiar de 0,5, sugerindo uma relação moderada entre as características e a qualidade do vinho.

Explorando as características de menor correlação, observamos que, em ambos os conjuntos de dados, o açúcar residual exibe a menor correlação. Além disso, na análise de Spearman, os sulfatos demonstram uma correlação fraca, enquanto na análise de Pearson, o pH é a variável com a correlação mais baixa.

Essas descobertas fornecem insights valiosos sobre os fatores que afetam a qualidade do vinho, informando futuras investigações e aprimorando nossa compreensão das relações entre as características químicas e organolépticas dessa bebida apreciada.

In [ ]:
df.corr(method = "spearman", numeric_only = True)['qualidade'].sort_values().plot(kind = 'barh')

In [ ]:
df.corr(method = "pearson", numeric_only = True)['qualidade'].sort_values().plot(kind = 'barh')

In [ ]:
sns.boxplot(data = df,
           x = "qualidade",
           y = "alcool")

# Análise univariada

In [ ]:
df.describe(include='all')

## Análise de Relação entre Notas e Teor Alcoólico de Vinhos

Nesta análise, investigamos a relação entre a pontuação de vinhos e seu teor alcoólico. Os resultados revelam uma tendência notável: vinhos com classificação superior (9) tendem a apresentar teores alcoólicos mais elevados. No entanto, é crucial observar que a presença de um alto teor alcoólico não é, por si só, um indicativo infalível de qualidade, visto que identificamos casos atípicos de vinhos com uma pontuação de 5 que exibem elevado teor alcoólico.


In [ ]:
sns.kdeplot(data = df,
           x = "alcool",
           hue = "qualidade",
           common_norm = False)

# Análise das Relações entre Classificação de Vinhos e Teor Alcoólico

Neste estudo, exploramos as relações entre a classificação de vinhos e seu teor alcoólico com base em dados empíricos. O gráfico apresentado acima ilustra as conclusões extraídas dessa análise.

## Resultados da Análise

O gráfico confirma de forma convincente a existência de uma correlação entre a classificação dos vinhos e seu teor alcoólico. Em particular, destacamos as seguintes descobertas:

- Vinhos com classificações mais elevadas, variando entre 7 e 9, exibem um teor alcoólico superior. Isso é evidenciado pela presença de um pico notável na densidade de amostras de vinho, concentrado entre 11 e 13 graus de álcool.

- Por outro lado, os vinhos com classificações mais baixas apresentam uma densidade de amostras mais significativa em uma faixa de graduação alcoólica entre 9 e 10.

Esses resultados sugerem que o teor alcoólico pode ser um fator relevante na classificação de vinhos, com vinhos mais alcoólicos geralmente recebendo classificações mais altas. Essa associação entre classificação e teor alcoólico pode ser de interesse para produtores, apreciadores e entusiastas de vinho que desejam compreender melhor os fatores subjacentes à apreciação de vinhos.

É importante notar que essa análise se baseia em dados disponíveis e pode não capturar todas as nuances do mundo dos vinhos. No entanto, os resultados aqui apresentados oferecem uma visão valiosa das relações entre classificação de vinhos e teor alcoólico, proporcionando um ponto de partida para investigações mais aprofundadas e tomada de decisões informadas.


### Os historiogramas são úteis para entender a distribuição de dados, identificar tendências, padrões e outliers em um conjunto de dados. 

In [ ]:
fig, axes = plt.subplots(5, 5)
colunas = ['acidez_fixa', 'acidez_volatil', 'acido_citrico', 'acucar_residual',
           'cloreto', 'dioxido_enxofre_livre', 'dioxido_enxofre_total',
           'densidade', 'pH', 'sulfatos', 'alcool', 'qualidade']
for i in range(5):
    for j in range(5):
        axes[i, j].hist(x=colunas[i + j], data=df,
                        edgecolor='#000000', linewidth=2, color='#ff4125')
        axes[i, j].set_title('Variação de ' + colunas[i + j])
fig = plt.gcf()
fig.set_size_inches(18, 18)
fig.tight_layout()


### Correlação entre diferentes características usando um mapa de correlação

In [ ]:
matriz_correlacao = df.corr()
mascara = np.array(matriz_correlacao)
mascara[np.tril_indices_from(mascara)] = False
figura = plt.gcf()
figura.set_size_inches(30, 12)

# Usar a paleta de cores "Reds" para criar um degradê de tons de vermelho
sns.heatmap(data=matriz_correlacao, mask=mascara, square=True, annot=True, cbar=True, cmap="Reds")


**INFERÊNCIAS DO MAPA DE CALOR ACIMA:**

1. A qualidade do vinho está altamente relacionada à acidez volátil.

2. Além disso, a qualidade do vinho está altamente correlacionada com o teor alcoólico.

3. O pH e o ácido cítrico/ acidez fixa estão altamente inversamente relacionados, pois todos sabemos que os ácidos têm valores de pH menores.

4. A relação consigo mesma (ou seja, de uma característica consigo mesma) é igual a 1, como esperado.

5. Outras inferências semelhantes podem ser feitas.

### Agora é possível visualizar a variação da qualidade em relação a distintos atributos numéricos.

#### A função cria e exibe três gráficos em uma única chamada, usando os parâmetros fornecidos. É útil quando desejar visualizar diferentes aspectos da relação entre duas variáveis em um conjunto de dados.

In [ ]:
def plot(feature_x, target='qualidade'):
    sns.catplot(x=target, y=feature_x, data=df, kind='bar', height=5, aspect=1)
    sns.catplot(x=target, y=feature_x, data=df, kind='violin', height=5, aspect=1)
    sns.catplot(x=target, y=feature_x, data=df, kind='swarm', height=5, aspect=1)

In [ ]:
plot('alcool','qualidade')

## Preparação Concluída: Avançando para a Modelagem de Dados

In [ ]:
intervalos = (2, 6.5, 8)
nomes_grupos = ['ruim', 'bom']
df['qualidade'] = pd.cut(df['qualidade'], bins=intervalos, labels=nomes_grupos)

In [ ]:
# Cria uma instância do LabelEncoder
codificador_rotulos = LabelEncoder()

##### Uma explicação sucinta sobre o LabelEncoder: Esta classe, parte da biblioteca scikit-learn (sklearn), é amplamente empregada para converter rótulos (categorias) em variáveis categóricas em valores numéricos. Essa conversão é essencial ao treinar modelos de aprendizado de máquina que exigem entradas numéricas em vez de categóricas. Por exemplo, permite transformar rótulos de classes, como "ruim" e "bom", em valores 0 e 1, tornando-os adequados como entradas para modelos de machine learning.

In [ ]:
# "Ruim" se torna 0 e "Bom" se torna 1
df['qualidade'] = codificador_rotulos.fit_transform(df['qualidade'])


**Divisão de Dados para Treinamento e Teste:**

Este código divide um conjunto de dados em duas partes: uma para treinar um modelo de machine learning (conjunto de treinamento) e outra para testar o modelo (conjunto de teste). Ele separa as características do conjunto de destino (rótulos) e permite avaliar o desempenho do modelo ao usar dados que não foram usados no treinamento.

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(
    df.drop('qualidade', axis=1), df['qualidade'], test_size=0.25, random_state=42)


## Comparação de Modelos de Classificação e Cálculo da Acurácia

In [ ]:
# Lista de modelos de classificação
modelos = [LinearSVC(), SVC(kernel='rbf'), KNeighborsClassifier(), RandomForestClassifier(),
           DecisionTreeClassifier(), GradientBoostingClassifier()]


nomes_dos_modelos = ['LinearSVM','rbfSVM','KNearestNeighbors','RandomForestClassifier','DecisionTree',
             'GradientBoostingClassifier',]

acuracias = []  # Lista para armazenar as acurácias dos modelos
resultados = {}  # Dicionário para armazenar os resultados

for i, modelo in enumerate(modelos):
    clf = modelo  # clf é uma abreviação comum para "classificador"
    clf.fit(x_treino, y_treino)  # Treina o classificador com os dados de treinamento
    previsoes = clf.predict(x_teste)  # Faz previsões com o conjunto de teste
    acuracia = accuracy_score(previsoes, y_teste)  # Calcula a acurácia das previsões
    acuracias.append(acuracia)  # Adiciona a acurácia à lista de acurácias

# Preenche o dicionário de resultados com os nomes dos modelos e suas acurácias correspondentes
resultados['Algoritmo de Modelagem'] = nomes_dos_modelos
resultados['Acurácia'] = acuracias

resultados  # Retorna o dicionário de resultados

In [ ]:
acuracias_frame = pd.DataFrame(resultados)
acuracias_frame

In [ ]:
sns.barplot(y='Algoritmo de Modelagem',x='Acurácia',data=acuracias_frame)

In [ ]:
sns.catplot(x='Algoritmo de Modelagem',y='Acurácia',data=acuracias_frame, kind='point', height=4, aspect=3.5)

No contexto da análise de dados, é importante notar que, inicialmente, as características não foram submetidas a nenhum tipo de escala. Isso significa que essas características têm diferentes escalas, o que implica que seus valores estão em intervalos ou unidades distintas. Agora, com o objetivo de entender melhor o impacto disso, a próxima etapa é realizar a escala das características. O escalonamento de características é uma prática comum em ciência de dados, utilizada para padronizar as características em uma escala uniforme. Essa ação visa avaliar como essa padronização afeta os resultados e o desempenho do modelo ou da análise em questão.

In [ ]:
def avaliar_modelos(x_treino, x_teste, y_treino, y_teste, nome_escalonador):
    # Lista de modelos a serem avaliados
# Lista de modelos de classificação
    modelos = [LinearSVC(), SVC(kernel='rbf'), KNeighborsClassifier(), RandomForestClassifier(),
           DecisionTreeClassifier(), GradientBoostingClassifier()]
           
    resultados_acuracia = []

    # Avaliar cada modelo
    for modelo in modelos:
        clf = modelo
        clf.fit(x_treino, y_treino)
        previsoes = clf.predict(x_teste)
        resultados_acuracia.append(accuracy_score(previsoes, y_teste))

    # Armazenar os resultados no dataframe
    acuracias_frame[nome_escalonador] = np.array(resultados_acuracia)


In [ ]:
df.shape

In [ ]:
# Extrair o array de labels 'qualidade' do DataFrame
Y = df['qualidade'].to_numpy()

# Lista de escaladores a serem aplicados aos dados
escaladores = [MinMaxScaler(), StandardScaler()]

# Nomes descritivos para os resultados dos escaladores
nomes_escalonadores = ['Acuracia_Min_Max_Scaler', 'Acuracia_Standard_Scaler']

# Iterar através dos escaladores e avaliar os modelos
for escala in range(len(escaladores)):
    escalador = escaladores[escala]

    # Aplicar o escalador aos dados
    escalador.fit(df)
    df_escalado = escalador.transform(df)

    # Selecionar as características escaladas 
    X = df_escalado[:,0:6]

    # Reatribuir Y com os rótulos de 'qualidade'
    Y = df['qualidade'].to_numpy()

    # Dividir os dados em treino e teste
    x_treino, x_teste, y_treino, y_teste = train_test_split(
        X, Y, test_size=0.25, random_state=42)

    # Avaliar os modelos com o nome do escalador correspondente
    avaliar_modelos(x_treino, x_teste, y_treino,
                    y_teste, nomes_escalonadores[escala])


## Técnicas de Escalonamento no Desempenho de Modelos de Classificação

O código tem o objetivo de avaliar como diferentes técnicas de escalonamento afetam o desempenho de modelos de classificação em um conjunto de dados específico. Utilizando a função `avaliar_modelos`, o código examina vários modelos de classificação em dados de treinamento e teste, fornecendo resultados que permitem a comparação do desempenho desses modelos após a aplicação de técnicas de escalonamento, como Min-Max Scaling e Standard Scaling. Esse processo é fundamental para determinar qual abordagem de pré-processamento de dados é mais eficaz na otimização do desempenho dos modelos, desempenhando um papel crucial no desenvolvimento de soluções de machine learning de sucesso.

In [ ]:
acuracias_frame

Neste momento, apresenta-se de forma clara as precisões de diferentes algoritmos de modelagem ao utilizar diferentes escaladores.

1. É importante notar que, neste contexto, as precisões aumentam marginalmente ao realizar o escalonamento.

2. Adicionalmente, para esses dados, o StandardScaling parece proporcionar resultados ligeiramente superiores em comparação com o MinMaxScaling.

3. Para alguns algoritmos de modelagem, observa-se um aumento significativo nas precisões ao escalonar as características, como SVM e KNN, enquanto, para outros, não se observa um aumento considerável nas precisões ao realizar o escalonamento.

In [ ]:
# código cria um gráfico de barras que ajuda a comparar a precisão de vários algoritmos de modelagem

sns.barplot(y='Algoritmo de Modelagem',x='Acurácia',data=acuracias_frame)

In [ ]:
sns.barplot(y='Algoritmo de Modelagem',x='Acuracia_Min_Max_Scaler',data=acuracias_frame)

In [ ]:
sns.barplot(y='Algoritmo de Modelagem',x='Acuracia_Standard_Scaler',data=acuracias_frame)

In [ ]:
# Extrai a coluna 'qualidade' como o alvo (variável de saída) Y e converte-a em um array NumPy.
Y = df['qualidade'].to_numpy()

# Prepara as features (variáveis de entrada) utilizando um StandardScaler para padronização dos dados, uma vez que isso produziu resultados melhores no passado.
escalador = StandardScaler()
df_escalado = escalador.fit_transform(df)
X = df_escalado[:, 0:11]  # Seleciona as colunas de 0 a 10 como as features padronizadas.

# Reatribui Y para garantir que estamos usando a variável alvo correta.
Y = df['qualidade'].to_numpy()

# Divide os dados em conjuntos de treinamento e teste usando a função train_test_split.
# 75% dos dados serão usados para treinamento (x_treino, y_treino) e 25% para teste (x_teste, y_teste).
# O valor de random_state é definido como 42 para garantir a reprodutibilidade dos resultados.
x_treino, x_teste, y_treino, y_teste, = train_test_split(X, Y, test_size=0.25, random_state=42)


## Ajuste de parâmetros

###### 1.   LOGISTIC REGRESSION.

In [ ]:
# Definindo uma lista de valores para os hiperparâmetros C e penalty
parametros_lr = {'C': [0.001, 0.01, 0.1, 1,
                    10, 100, 1000], 'penalty': ['l1', 'l2']}

# Criando um estimador de Regressão Logística para otimização de hiperparâmetros
classificador_lr = GridSearchCV(estimator=LogisticRegression(),
                      param_grid=parametros_lr, scoring='accuracy', cv=10)

# Ajustando (treinando) o modelo com os dados de treinamento x_treino e y_treino
classificador_lr.fit(x_treino, y_treino)


In [ ]:
# Obtém os melhores hiperparâmetros encontrados durante a otimização.
classificador_lr.best_params_

In [ ]:
# Obtém a melhor pontuação (geralmente acurácia) alcançada com os melhores hiperparâmetros.
classificador_lr.best_score_


In [ ]:
# Acessa os resultados completos da validação cruzada, incluindo as métricas para cada combinação de hiperparâmetros.
classificador_lr.cv_results_

In [ ]:
# Faz previsões com o modelo otimizado nos dados de teste.
previsoes = classificador_lr.predict(x_teste)

# Calcula a acurácia das previsões comparando com os valores reais nos dados de teste.
accuracy_score(previsoes, y_teste)

######   2. KNN.

In [ ]:
# Cria uma lista de valores para o número de vizinhos
lista_vizinhos = [i + 1 for i in range(50)]

# Define um dicionário de parâmetros para o KNeighborsClassifier
parametros_knn = {'n_neighbors': lista_vizinhos, 'n_jobs': [-1]}

# Cria um classificador KNeighborsClassifier com busca em grade
classificador_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=parametros_knn, scoring='accuracy', cv=10)

# Treina o classificador com os dados de treinamento
classificador_knn.fit(x_treino,y_treino)


In [ ]:
classificador_knn.best_score_

In [ ]:
classificador_knn.best_params_

In [ ]:
classificador_knn.cv_results_

In [ ]:
# Faz previsões com o modelo otimizado nos dados de teste.
previsoes = classificador_knn.predict(x_teste)

# Calcula a acurácia das previsões comparando com os valores reais nos dados de teste.
accuracy_score(previsoes, y_teste)

######   3. RANDOM FOREST.

In [ ]:
# Definição dos parâmetros para o Random Forest
parametros_rf = {'n_estimators': [500],  # Número de árvores na floresta
                 'max_features': ['auto', 'sqrt', 'log2']}  # Método de seleção de características

# Criação do classificador Random Forest com Grid Search Cross-Validation
classificador_rf = GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1),
                                param_grid=parametros_rf,  # Parâmetros a serem ajustados
                                scoring='accuracy',  # Métrica de avaliação
                                cv=10)  # Validação cruzada com 10 folds

# Treinamento do classificador Random Forest
classificador_rf.fit(x_treino, y_treino)  # Dados de treinamento


In [ ]:
classificador_rf.best_score_

In [ ]:
classificador_rf.best_params_

In [ ]:
classificador_rf.cv_results_

In [ ]:
previsoes = classificador_rf.predict(x_teste)
accuracy_score(previsoes,y_teste)

###### 4. GRADIENT BOOSTING.

In [ ]:
# Inicializando o classificador Gradient Boosting com GridSearchCV.
classificador_gb = GridSearchCV(estimator=GradientBoostingClassifier(
), cv=10, param_grid={'n_estimators': [500]})

# Treinando o classificador Gradient Boosting com os dados de treinamento.
classificador_gb.fit(x_treino, y_treino)


In [ ]:
classificador_gb.best_score_

In [ ]:
classificador_gb.best_params_

In [ ]:
classificador_gb.cv_results_

In [ ]:
previsoes = classificador_gb.predict(x_teste)
accuracy_score(previsoes,y_teste)

Apoś o ajuste de parâmetros , o SVM com o kernel RBF alcançou a mais alta precisão, que foi de 91,75%.